## BONUS: CJA Data View Solution Design Reference Generator

### Overview

The below Python code is a comprehensive script for generating a Customer Journey Analytics (CJA) Solution Design reference.

It starts by importing the necessary libraries and initializing the CJA object. It then specifies the data view ID and retrieves all available metrics and dimensions related to this data view.

The code proceeds to perform a data view lookup, ensuring all values are iterable and all arrays have the same length for consistency.

The resulting data is then transformed into a pandas DataFrame, which is written to an Excel file. The Excel file includes separate sheets for the data view, metrics, and dimensions.

The final part of the code applies formatting to the Excel sheets for better readability, alternating row colors between grey and white.

This script is a valuable tool for any CJA administrator or practitioners working with CJA, as it automates the process of generating a detailed Solution Design reference.

In [7]:
import cjapy
import pandas as pd

# Instantiate cjapy
cjapy.importConfigFile("python_config.json")
cja = cjapy.CJA()

# Set the Data View id we want into a variable
data_view = "dv_62ba17d5a5d7845496f5fb4d"

# Grab all available CJA metrics
metrics = cja.getMetrics(data_view, inclType=True, full=True)

# Grab all available CJA dimensions
dimensions = cja.getDimensions(data_view, inclType=True, full=True)

# Perform a CJA data view lookup using getDataView module
lookup_data = cja.getDataView(data_view)

# Ensure all values are iterable
for key, value in lookup_data.items():
    if not isinstance(value, (list, tuple)):
        lookup_data[key] = [value]

# Ensure all arrays have the same length
max_length = max(len(value) for value in lookup_data.values())
for key, value in lookup_data.items():
    if len(value) < max_length:
        lookup_data[key] += [None] * (max_length - len(value))

# Convert dictionary to DataFrame
lookup_df = pd.DataFrame(lookup_data)

# Create an Excel writer object with CJA data view name and data view id included in the file name
excel_file_name = 'CJA_DataView_' + str(lookup_data["name"]) + '_' + str(data_view) + '_SDR.xlsx'
with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:


    # Write lookup data to first tab
    lookup_df.to_excel(writer, sheet_name='DataView', index=False)

    # Write metrics data to the second tab
    metrics.to_excel(writer, sheet_name='Metrics', index=False)
    
    # Get the workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Metrics']
    
    # Define formats for grey and white rows
    grey_format = workbook.add_format({'bg_color': '#F2F2F2'})
    white_format = workbook.add_format({'bg_color': '#FFFFFF'})
    
    # Apply grey and white formatting to Metrics sheet
    for idx, row in enumerate(metrics.iterrows()):
        if idx % 2 == 0:
            worksheet.set_row(idx+1, cell_format=grey_format)
        else:
            worksheet.set_row(idx+1, cell_format=white_format)
    
    # Write dimensions data to the third tab
    dimensions.to_excel(writer, sheet_name='Dimensions', index=False)
    
    # Get the worksheet object for Dimensions sheet
    worksheet = writer.sheets['Dimensions']
    
    # Apply grey and white formatting to Dimensions sheet
    for idx, row in enumerate(dimensions.iterrows()):
        if idx % 2 == 0:
            worksheet.set_row(idx+1, cell_format=grey_format)
        else:
            worksheet.set_row(idx+1, cell_format=white_format)
